In [0]:

dbutils.secrets.help()
dbutils.secrets.listScopes()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

[SecretScope(name='demoscope')]

In [0]:
SecretID=dbutils.secrets.get(scope='demoscope',key='SecretID')
ClientID=dbutils.secrets.get(scope='demoscope',key='ClientID')
TenantID=dbutils.secrets.get(scope='demoscope',key='TenantID')

spark.conf.set("fs.azure.account.auth.type.indianfoodadls.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.indianfoodadls.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.indianfoodadls.dfs.core.windows.net",ClientID )
spark.conf.set("fs.azure.account.oauth2.client.secret.indianfoodadls.dfs.core.windows.net", SecretID)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.indianfoodadls.dfs.core.windows.net", f"https://login.microsoftonline.com/{TenantID}/oauth2/token")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


spark = SparkSession.builder.appName("ReadData").getOrCreate()

# Sample code to load data

df = spark.read.csv("abfss://raw@indianfoodadls.dfs.core.windows.net/indian_food.csv", header=True, inferSchema=True)
df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [0]:
#Renaming the column names
result = df.withColumnRenamed("age", "Age") \
    .withColumnRenamed("name", "Name") \
    .withColumnRenamed("ingredients", "Ingredients") \
    .withColumnRenamed("diet", "Diet") \
    .withColumnRenamed("prep_time", "Prep_Time") \
    .withColumnRenamed("cook_time", "Cook_Time") \
    .withColumnRenamed("flavor_profile", "Flavor_Profile")\
    .withColumnRenamed("course", "Course")\
    .withColumnRenamed("state", "State")\
    .withColumnRenamed("region", "Region")

result.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          Name|         Ingredients|      Diet|Prep_Time|Cook_Time|Flavor_Profile| Course|        State|Region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [0]:
result.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Ingredients: string (nullable = true)
 |-- Diet: string (nullable = true)
 |-- Prep_Time: integer (nullable = true)
 |-- Cook_Time: integer (nullable = true)
 |-- Flavor_Profile: string (nullable = true)
 |-- Course: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Region: string (nullable = true)



In [0]:
#Dropping the duplicates

result = result.dropDuplicates()
result.show()

+--------------------+--------------------+--------------+---------+---------+--------------+-----------+------------+----------+
|                Name|         Ingredients|          Diet|Prep_Time|Cook_Time|Flavor_Profile|     Course|       State|    Region|
+--------------------+--------------------+--------------+---------+---------+--------------+-----------+------------+----------+
|               Kheer|Milk, rice, sugar...|    vegetarian|       10|       40|         sweet|    dessert|          -1|        -1|
|               Puttu|Brown rice flour,...|    vegetarian|      495|       40|            -1|main course|      Kerala|     South|
|      Gajar ka halwa|Carrots, milk, su...|    vegetarian|       15|       60|         sweet|    dessert|      Punjab|     North|
|          Thalipeeth|Whole wheat flour...|    vegetarian|       25|       30|         spicy|main course| Maharashtra|      West|
|          Farsi Puri|Semolina, clarifi...|    vegetarian|       -1|       -1|            

In [0]:
# Adding a new column that calculates the total time required to prepare a dish.

result = result.withColumn("Total_time", col("Prep_time") + col("Cook_time"))
result.show(5)

+--------------+--------------------+----------+---------+---------+--------------+-----------+-----------+------+----------+
|          Name|         Ingredients|      Diet|Prep_Time|Cook_Time|Flavor_Profile|     Course|      State|Region|Total_time|
+--------------+--------------------+----------+---------+---------+--------------+-----------+-----------+------+----------+
|         Kheer|Milk, rice, sugar...|vegetarian|       10|       40|         sweet|    dessert|         -1|    -1|        50|
|         Puttu|Brown rice flour,...|vegetarian|      495|       40|            -1|main course|     Kerala| South|       535|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|    dessert|     Punjab| North|        75|
|    Thalipeeth|Whole wheat flour...|vegetarian|       25|       30|         spicy|main course|Maharashtra|  West|        55|
|    Farsi Puri|Semolina, clarifi...|vegetarian|       -1|       -1|            -1|      snack|    Gujarat|  West|    

In [0]:
# Grouping the data by Region and calculate the average preparation and cooking time.

region_time_df = result.groupBy("state").agg(
    avg("Prep_time").alias("Avg_Prep_Time"),
    avg("Cook_time").alias("Avg_Cook_Time")
)

region_time_df.show()

+---------------+------------------+------------------+
|          state|     Avg_Prep_Time|     Avg_Cook_Time|
+---------------+------------------+------------------+
|       Nagaland|               5.0|              15.0|
|      Karnataka|60.833333333333336|44.166666666666664|
|             -1|63.083333333333336|35.041666666666664|
|         Odisha|12.571428571428571|37.714285714285715|
|         Kerala|             71.75|             29.25|
|     Tamil Nadu|             14.35|             22.85|
|   Chhattisgarh|              10.0|              60.0|
| Andhra Pradesh|              61.0|              37.5|
| Madhya Pradesh|              15.0|              42.5|
|         Punjab|             48.75|          37.34375|
|        Manipur|             119.5|              22.0|
|Jammu & Kashmir|               9.5|              24.5|
|            Goa| 9.666666666666666|              33.0|
|        Haryana|              10.0|              60.0|
|        Gujarat|13.742857142857142|26.485714285

In [0]:

result = result.fillna({"flavor_profile": "Unknown", "State": "Unknown"})
result.show(50)

+--------------------+--------------------+--------------+---------+---------+--------------+-----------+---------------+----------+----------+
|                Name|         Ingredients|          Diet|Prep_Time|Cook_Time|Flavor_Profile|     Course|          State|    Region|Total_time|
+--------------------+--------------------+--------------+---------+---------+--------------+-----------+---------------+----------+----------+
|               Kheer|Milk, rice, sugar...|    vegetarian|       10|       40|         sweet|    dessert|             -1|        -1|        50|
|               Puttu|Brown rice flour,...|    vegetarian|      495|       40|            -1|main course|         Kerala|     South|       535|
|      Gajar ka halwa|Carrots, milk, su...|    vegetarian|       15|       60|         sweet|    dessert|         Punjab|     North|        75|
|          Thalipeeth|Whole wheat flour...|    vegetarian|       25|       30|         spicy|main course|    Maharashtra|      West|    

In [0]:
# Sort the data by total preparation time

result = result.orderBy("Total_time", ascending=False)
result.show(7)

+-----------+--------------------+----------+---------+---------+--------------+-----------+-----------+------+----------+
|       Name|         Ingredients|      Diet|Prep_Time|Cook_Time|Flavor_Profile|     Course|      State|Region|Total_time|
+-----------+--------------------+----------+---------+---------+--------------+-----------+-----------+------+----------+
|  Shrikhand|Curd, sugar, saff...|vegetarian|       10|      720|         sweet|    dessert|Maharashtra|  West|       730|
|Pindi chana|Fennel, tea bags,...|vegetarian|      500|      120|         spicy|main course|     Punjab| North|       620|
|      Puttu|Brown rice flour,...|vegetarian|      495|       40|            -1|main course|     Kerala| South|       535|
|  Misti doi|       Milk, jaggery|vegetarian|      480|       30|         sweet|    dessert|West Bengal|  East|       510|
|       Dosa|Chana dal, urad d...|vegetarian|      360|       90|         spicy|      snack|         -1| South|       450|
|Masala Dosa|Cha

In [0]:
# Pivot the data to show the number of dishes per course for each region
pivot_df = result.groupBy("Region").pivot("Course").count()
pivot_df.show()

+----------+-------+-----------+-----+-------+
|    Region|dessert|main course|snack|starter|
+----------+-------+-----------+-----+-------+
|        -1|      5|          4|    4|   NULL|
|      NULL|      1|       NULL| NULL|   NULL|
|     South|     18|         31|   10|   NULL|
|   Central|      1|          1|    1|   NULL|
|      East|     21|         10| NULL|   NULL|
|      West|     23|         28|   23|   NULL|
|North East|      7|         18| NULL|   NULL|
|     North|      9|         37|    1|      2|
+----------+-------+-----------+-----+-------+



In [0]:
pivot_df.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/pivot_transformed")
result.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/result")
region_time_df.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/region_time")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2380092116466689>, line 1
----> 1 pivot_df.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/pivot_transformed")
      2 result.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/result")
      3 region_time_df.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@indianfoodadls.dfs.core.windows.net/region_time")

NameError: name 'pivot_df' is not defined